In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import joblib
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow import keras
#from sklearn.preprocessing import StandardScaler


%store -r X_features_full
%store -r X_features_poly_full
%store -r y_labels_full

%store -r totaldf_full
totaldf = totaldf_full
X_features = X_features_full
X_features_poly = X_features_poly_full
y_labels = y_labels_full

In [2]:
voting_reg = joblib.load("voting_reg.pkl")
poly_ridge_reg = joblib.load("poly_ridge_reg.pkl")
svm_rbf_reg = joblib.load("svm_rbf_reg.pkl")
forest_reg = joblib.load("forest_regressor.pkl")
seq_model = keras.models.load_model('D:\python\\full data\\first')



In [3]:
totaldf.insert(7,'poly ridge μ', totaldf.μob*(np.exp(np.power(poly_ridge_reg.predict(X_features_poly[2]),2)) -0.0012))
totaldf.insert(8,'rbf μ', totaldf.μob*(np.exp(np.power(svm_rbf_reg.predict(X_features[2]),2)) -0.0012))
totaldf.insert(9,'forest μ', totaldf.μob*(np.exp(np.power(forest_reg.predict(X_features[2]),2)) -0.0012))
totaldf.insert(10,'voting μ', totaldf.μob*(np.exp(np.power(voting_reg.predict(X_features[2]),2)) -0.0012))
totaldf.insert(11,'sequential', (np.exp(np.power(seq_model.predict(X_features[2]),2)) -0.0012))
totaldf.sequential = totaldf.sequential*totaldf.μob
totaldf.insert(12,'Almehaideb', totaldf.μob*np.power(totaldf.P/totaldf.Pb,0.134819+1.94345*np.power(10,-4.0)*totaldf.Rsob-1.93106*np.power(10,-9.0)*np.power(totaldf.Rsob,2)))
X = 0.776644115+0.987658646*np.log10(totaldf.μob)-0.190564677*np.log10(totaldf.Rsob)+0.009147711*totaldf.μob*np.log10(totaldf.Rsob)-0.000019111*(totaldf.P-totaldf.Pb)

totaldf.insert(13,'Dindoruk and Christman',totaldf.μob +0.00006334*totaldf.ΔP*np.power(10,X))

72/72 [==============================] - 0s 1ms/step


In [42]:
def Stats_matrix(a):
    absolute_average_error = []
    for i in range(7):
        absolute_average_error.append(np.average(np.absolute((a.iloc[:,i+7]-a.μ)/a.μ*100)))
    statistics  = pd.Series(absolute_average_error)
    return statistics

Stats_matrix(totaldf)

0    3.330969
1    3.142564
2    3.057571
3    3.050594
4    3.310500
5    7.207309
6    6.781396
dtype: float64

In [43]:
y = Stats_matrix(totaldf)

In [47]:
y.values.tolist()

[3.33096942124699,
 3.1425644536963255,
 3.057570817645391,
 3.0505939684710857,
 3.3104998648975585,
 7.207309452538061,
 6.78139588388417]

In [48]:
y

0    3.330969
1    3.142564
2    3.057571
3    3.050594
4    3.310500
5    7.207309
6    6.781396
dtype: float64

AttributeError: 'list' object has no attribute 'type'